In [51]:
# How to Run Django in Jupyter Notebook
# https://www.youtube.com/watch?v=TN5zIA--cz4&ab_channel=TechSunami

In [52]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db import connection
from django.db import reset_queries
from django.db import models
from django_orm_app.models import Author, Blog
from faker import Faker 
import random


In [53]:
def define_model(model):
    print(f"----------------- {model.__name__}-------------------")
    for field in model._meta.fields:
        print(f"{field.name} --> {field.get_internal_type()}")
    print('\033[1m' + f"Table name in database is {model._meta.db_table}" + '\033[0m')

In [54]:
define_model(Author)
define_model(Blog)

----------------- Author-------------------
id --> BigAutoField
name --> CharField
email --> CharField
age --> IntegerField
creation_date --> DateTimeField
updation_date --> DateTimeField
Table name in database is author_model
----------------- Blog-------------------
id --> BigAutoField
name --> CharField
author --> ForeignKey
time_to_read --> IntegerField
tagline --> TextField
creation_date --> DateTimeField
updation_date --> DateTimeField
Table name in database is blog_model


### Generating fake data

In [55]:
# fake = Faker()
# range_of_author_id = 100

In [56]:
# lst_author = []
# for _ in range(range_of_author_id):
#     lst_author.append(
#         Author(
#             name=fake.name(), 
#             age=random.randint(1,100)
#         )
#     )
# objs_author = Author.objects.bulk_create(lst_author)

In [57]:
# lst_blog = []
# for _ in range(10000):
#     lst_blog.append(
#         Blog(
#             name=fake.sentence(), 
#             tagline= fake.paragraph(), 
#             time_to_read = random.randint(5, 60),
#             author_id=random.randint(1, 100)
#         )
#     )
# objs_blog = Blog.objects.bulk_create(lst_blog)

## Table of Content
1. [Select Related](#1)
    1. [Case 1](#1.a) 
2. [Prefetch Related](#2)
    1. [Case 1](#2.a) 
3. [update_fields](#3)
    1. [Case 1](#3.a)
    2. [Case 2](#3.b)
4. [F()](#4)
    1. [Case 1](#4.a)
    2. [Case 2](#4.b)

In [58]:
def get_query():
    return connection.queries

<a id="1"></a>
### Select Related
<pre>One uses select_related when the object that you’re going to be selecting is a single object, so OneToOneField or a ForeignKey</pre>
Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#select-related

<a id="1.a"></a>
<h5><b>Case 1</b> - We want author name of blog having id = 1</h5>

In [59]:
#Not optimized
reset_queries()
blog_obj = Blog.objects.get(id=1)
print(blog_obj.author.name)
get_query()


Vanessa Wilkinson


[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."id" = 1 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 63 LIMIT 21',
  'time': '0.001'}]

In [61]:
#optimized
reset_queries()
blog_obj = Blog.objects.select_related("author").get(id=1)
print(blog_obj.author.name)
get_query()


Vanessa Wilkinson


[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date", "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "blog_model" LEFT OUTER JOIN "author_model" ON ("blog_model"."author_id" = "author_model"."id") WHERE "blog_model"."id" = 1 LIMIT 21',
  'time': '0.001'}]

<a id="1.b"></a>
<h5><b>Case 2</b> - Get all author name whose blog take time to read more than 50 mins</h5>

In [62]:
#Not optimized
reset_queries()
blog_queryset = Blog.objects.filter(time_to_read__gt=50)[:5]
for obj in blog_queryset:
    print(obj.author.name)
get_query()

Sylvia Kelly
Mr. Danny Rhodes
Angela Potts
Robert Duran
Lisa Adams


[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."time_to_read" > 50 LIMIT 5',
  'time': '0.001'},
 {'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 34 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 86 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id

In [64]:
#optimized
reset_queries()
blog_queryset = Blog.objects.filter(time_to_read__gt=50)[:5].select_related("author")
for obj in blog_queryset:
    print(obj.author.name)
get_query()

Sylvia Kelly
Mr. Danny Rhodes
Angela Potts
Robert Duran
Lisa Adams


[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date", "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "blog_model" LEFT OUTER JOIN "author_model" ON ("blog_model"."author_id" = "author_model"."id") WHERE "blog_model"."time_to_read" > 50 LIMIT 5',
  'time': '0.000'}]

<a id="2"></a>
### Prefetch Related
<pre>You use prefetch_related when you’re going to get a “set” of things, so ManyToManyFields as you stated or reverse ForeignKeys.”</pre>
Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#prefetch-related

<h5><b>Case 1</b> - We want top 10 blog of author having id = 1</h5>

In [65]:
#Not optimized
reset_queries()
author_obj = Author.objects.get(id=1)
blog_queryset = author_obj.author_blog.all()[:10]
print(blog_queryset)
print("-------------")
for obj in blog_queryset:
    print(obj)
get_query()

<QuerySet [<Blog: Include dinner tonight production under conference student anything.>, <Blog: Wall town pull feeling near miss vote money.>, <Blog: Site western issue wish herself section successful.>, <Blog: His memory expert rather increase either.>, <Blog: Argue stay cut between series collection.>, <Blog: Decade knowledge politics action law maintain month.>, <Blog: Pick ball evidence support region discussion.>, <Blog: Medical difference build group attorney remember.>, <Blog: Fire stop after beautiful day.>, <Blog: Beautiful play discover former once.>]>
-------------
Include dinner tonight production under conference student anything.
Wall town pull feeling near miss vote money.
Site western issue wish herself section successful.
His memory expert rather increase either.
Argue stay cut between series collection.
Decade knowledge politics action law maintain month.
Pick ball evidence support region discussion.
Medical difference build group attorney remember.
Fire stop after be

[{'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 1 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."author_id" = 1 LIMIT 10',
  'time': '0.000'},
 {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."author_id" = 1 LIMIT 10',
  'time': '0.000'}]

In [66]:
# optimized
reset_queries()
author_obj = Author.objects.prefetch_related("author_blog").get(id=1)
blog_queryset = author_obj.author_blog.all()[:10]
print(blog_queryset)
print("-------------")
for obj in blog_queryset:
    print(obj)
get_query()

[<Blog: Include dinner tonight production under conference student anything.>, <Blog: Wall town pull feeling near miss vote money.>, <Blog: Site western issue wish herself section successful.>, <Blog: His memory expert rather increase either.>, <Blog: Argue stay cut between series collection.>, <Blog: Decade knowledge politics action law maintain month.>, <Blog: Pick ball evidence support region discussion.>, <Blog: Medical difference build group attorney remember.>, <Blog: Fire stop after beautiful day.>, <Blog: Beautiful play discover former once.>]
-------------
Include dinner tonight production under conference student anything.
Wall town pull feeling near miss vote money.
Site western issue wish herself section successful.
His memory expert rather increase either.
Argue stay cut between series collection.
Decade knowledge politics action law maintain month.
Pick ball evidence support region discussion.
Medical difference build group attorney remember.
Fire stop after beautiful day

[{'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 1 LIMIT 21',
  'time': '0.001'},
 {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."author_id" IN (1)',
  'time': '0.000'}]

<a id="3"></a>
### update_fields

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/instances/#ref-models-update-fields

<a id="3.a"></a>
<h5><b>Case </b> - Change tagline of blog id = 5</h5>

In [67]:
#Not optimized
reset_queries()
blog_obj = Blog.objects.get(id=5)
blog_obj.tagline = "New Tagline"
blog_obj.save()
get_query()

[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."id" = 5 LIMIT 21',
  'time': '0.001'},
 {'sql': 'UPDATE "blog_model" SET "name" = \'Natural deal attention defense oil.\', "author_id" = 34, "time_to_read" = 51, "tagline" = \'New Tagline\', "creation_date" = \'2023-07-08 14:46:17.789535\', "updation_date" = \'2023-07-08 20:52:32.372212\' WHERE "blog_model"."id" = 5',
  'time': '0.001'}]

In [15]:
# optimized
reset_queries()
blog_obj = Blog.objects.get(id=5)
blog_obj.tagline = "New Tagline"
blog_obj.save(update_fields=["tagline"])
get_query()

[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."id" = 5 LIMIT 21',
  'time': '0.005'},
 {'sql': 'UPDATE "blog_model" SET "tagline" = \'New Tagline\' WHERE "blog_model"."id" = 5',
  'time': '0.000'}]

In [16]:
# More optimized
reset_queries()
blog_obj = Blog.objects.only("tagline").get(id=5)
blog_obj.tagline = "New Tagline"
blog_obj.save(update_fields=["tagline"])
get_query()

[{'sql': 'SELECT "blog_model"."id", "blog_model"."tagline" FROM "blog_model" WHERE "blog_model"."id" = 5 LIMIT 21',
  'time': '0.000'},
 {'sql': 'UPDATE "blog_model" SET "tagline" = \'New Tagline\' WHERE "blog_model"."id" = 5',
  'time': '0.000'}]

<a id="4"></a>
### F( )

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/expressions/#f-expressions

<a id="4.a"></a>
<h5><b>Case 1</b> - Increase Age of Author having id = 1 by 1</h5>

In [68]:
#Not optimized
reset_queries()
author_obj = Author.objects.get(id=1)
print(author_obj.age)
author_obj.age += 1
author_obj.save()
get_query()

114


[{'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 1 LIMIT 21',
  'time': '0.000'},
 {'sql': 'UPDATE "author_model" SET "name" = \'Kathleen Hess\', "email" = NULL, "age" = 115, "creation_date" = \'2023-07-08 14:45:45.208582\', "updation_date" = \'2023-07-08 20:55:02.204255\' WHERE "author_model"."id" = 1',
  'time': '0.002'}]

In [69]:
from django.db.models import F
#optimized
reset_queries()
author_obj = Author.objects.get(id=1)
print(author_obj.age)
author_obj.age = F("age") + 1
author_obj.save()
get_query()

115


[{'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 1 LIMIT 21',
  'time': '0.001'},
 {'sql': 'UPDATE "author_model" SET "name" = \'Kathleen Hess\', "email" = NULL, "age" = ("author_model"."age" + 1), "creation_date" = \'2023-07-08 14:45:45.208582\', "updation_date" = \'2023-07-08 20:55:09.563813\' WHERE "author_model"."id" = 1',
  'time': '0.002'}]

In [70]:
from django.db.models import F
# More optimized
reset_queries()
Author.objects.filter(id=1).update(age = F("age")+1)
get_query()

[{'sql': 'UPDATE "author_model" SET "age" = ("author_model"."age" + 1) WHERE "author_model"."id" = 1',
  'time': '0.003'}]

<b>Another Advantage is Avoiding race condition using F()</b>

<a id="4.b"></a>
<h5><b>Case 2</b> - In bizzare world you want all blog whose time to read is less than author age 😂</h5>

In [75]:
# Not optimized
reset_queries()
for obj in Blog.objects.all()[:8]:
    if obj.time_to_read < obj.author.age:
        print(obj.time_to_read, obj.author.age)
get_query()

34 41
11 22
9 67
32 33
50 94


[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" LIMIT 8',
  'time': '0.000'},
 {'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 63 LIMIT 21',
  'time': '0.001'},
 {'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 91 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 39 LIMIT 21',
  'time': '0.000'},
 

In [72]:
from django.db.models import F
# optimized
reset_queries()
for obj in Blog.objects.filter(time_to_read__lt = F("author__age"))[:5]:
    print(obj.time_to_read, obj.author.age)
get_query()

34 41
11 22
9 67
32 33
50 94


[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" INNER JOIN "author_model" ON ("blog_model"."author_id" = "author_model"."id") WHERE "blog_model"."time_to_read" < ("author_model"."age") LIMIT 5',
  'time': '0.001'},
 {'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 63 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "author_model" WHERE "author_model"."id" = 91 LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_m

In [77]:
from django.db.models import F
# More optimized
reset_queries()
for obj in Blog.objects.filter(time_to_read__lt = F("author__age"))[:5].select_related("author"):
    print(obj.time_to_read, obj.author.age)
get_query()

34 41
11 22
9 67
32 33
50 94


[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."time_to_read", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date", "author_model"."id", "author_model"."name", "author_model"."email", "author_model"."age", "author_model"."creation_date", "author_model"."updation_date" FROM "blog_model" INNER JOIN "author_model" ON ("blog_model"."author_id" = "author_model"."id") WHERE "blog_model"."time_to_read" < ("author_model"."age") LIMIT 5',
  'time': '0.001'}]

## Hey.... If You are liked the content Do subscribe to my Youtube Channel
https://www.youtube.com/channel/UCpuKtSx_Cu8_ikyhlKAWBYA?sub_confirmation=1

<b>Also You can connect with me on Linkedin</b> - https://www.linkedin.com/in/lakshit-ukani/